# Problem statement:
This Project is having 2 Problems to solve.
This dataset has 11 columns and 10683 rows.our task one is to analyze the dataset and predict the flight ticket price by developing a supervised machine learning model.

In [68]:
# Import all the required liabrary
import numpy as np
import pandas as pd
import sklearn 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.feature_selection import RFE

In [69]:
#Load Data Set
df_train = pd.read_csv("Data_Train.csv")
df_test = pd.read_csv("Test_set.csv")
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


# Analyzing the data:
Let’s start analyzing the data provided. We need to know the number of columns, rows, null objects, etc. So, let’s open the data and start counting the columns, get the row count, go through each row to find out any special characters or null values in it, find out the data type of each column provided.

In [70]:
#check the shape of data
df_train.shape

(10683, 11)

Dataset is having 9134 Rows and 26 columns.

In [71]:
df_train.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [72]:
#Check if any missing value present in data
df_train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [73]:
#Check null column values
null_columns=df_train.columns[df_train.isnull().any()]
print(df_train[df_train.isnull().any(axis=1)][null_columns].head())

     Route Total_Stops
9039   NaN         NaN


In [74]:
#drop null column values
df_train.dropna(inplace=True)

In [75]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = df_train[df_train.duplicated()]
print("Total Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF.shape

Total Duplicate Rows except first occurrence based on all columns are :


(220, 11)

In [76]:
#remove duplicate rows in training dataset
df_train.drop_duplicates(keep='first',inplace=True)

In [77]:
#See values in columns of dataset 
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL ? BOM ? COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? MAA ? BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL ? BOM ? COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL ? BOM ? COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR ? DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [78]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


# Feature Engineering:
Machine Learning model requires input data in numerical notations to extract patterns from it and make predictions. But, not all the data provided in our source dataset is numerical. the data provided in object column is Categorical data. we need to convert these into numerical notations. Here data is nothing but a feature that our model uses as an input. So, we perform Feature Engineering on our data to create meaningful numerical data out of the source dataset.


In [79]:
print("Train set:\n",df_train["Additional_Info"].value_counts())
print("\nPrediction set:\n",df_test["Additional_Info"].value_counts())

Train set:
 No info                         8182
In-flight meal not included     1926
No check-in baggage included     318
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
2 Long layover                     1
Red-eye flight                     1
1 Short layover                    1
Name: Additional_Info, dtype: int64

Prediction set:
 No info                         2148
In-flight meal not included      444
No check-in baggage included      76
Business class                     1
Change airports                    1
1 Long layover                     1
Name: Additional_Info, dtype: int64


In [80]:
# replace repeating value in train set
df_train["Additional_Info"] = df_train["Additional_Info"].replace({'No Info': 'No info'})

In [81]:
df_test["Additional_Info"].value_counts()

No info                         2148
In-flight meal not included      444
No check-in baggage included      76
Business class                     1
Change airports                    1
1 Long layover                     1
Name: Additional_Info, dtype: int64

In [82]:
df_test["Destination"].value_counts()

Cochin       1145
Banglore      710
Delhi         317
New Delhi     238
Hyderabad     186
Kolkata        75
Name: Destination, dtype: int64

In [83]:
#Repplace Delhi with New Delhi
df_train["Destination"] = df_train["Destination"].replace({'New Delhi': 'Delhi'})
df_test["Destination"] = df_test["Destination"].replace({'New Delhi': 'Delhi'})

In [84]:
# Assign values manually to Total_stops column since label encoder might assign 
# wrong values. Like it can assign value 1 to 'non-stop' or 2 to '3 stops' 
df_train.Total_Stops.replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)
df_test.Total_Stops.replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)
df_train["Total_Stops"] = df_train["Total_Stops"].astype(int)
df_test["Total_Stops"] = df_test["Total_Stops"].astype(int)

In [85]:
# feature engineering on Date_of_Journey column
# Extract day and month of journey from the above column in both train and test dataset

df_train["isWeekend"] = ((pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.dayofweek) // 5 == 1).astype(int)
df_train["Day_of_Week"] = pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.day_name()
df_train["Day_Of_Journey"] = pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.day
df_train["Month_of_Journey"] = pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.month

df_test["isWeekend"] = ((pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.dayofweek) // 5 == 1).astype(int)
df_test["Day_Of_Journey"] = pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.day
df_test["Day_of_Week"] = pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.day_name()
df_test["Month_of_Journey"] = pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.month

In [86]:
# cleaning duration column in train dataset
duration = list(df_train["Duration"])

for i in range(len(duration)) :
    if len(duration[i].split()) != 2:
        if 'h' in duration[i] :
            duration[i] = duration[i].strip() + ' 0m'
        elif 'm' in duration[i] :
            duration[i] = '0h {}'.format(duration[i].strip())

dur_hours = []
dur_minutes = []  
dur_seconds = []
for i in range(len(duration)) :
    dur_minutes.append(int(duration[i].split()[0][:-1])*60 + int(duration[i].split()[1][:-1]))
    dur_seconds.append(int(duration[i].split()[0][:-1])*60*60 + int(duration[i].split()[1][:-1])*60)
df_train["Duration_minutes"] = dur_minutes
df_train.drop(["Duration"], axis=1, inplace = True)

In [87]:
# cleaning duration column in test dataset
duration = list(df_test["Duration"])

for i in range(len(duration)) :
    if len(duration[i].split()) != 2:
        if 'h' in duration[i] :
            duration[i] = duration[i].strip() + ' 0m'
        elif 'm' in duration[i] :
            duration[i] = '0h {}'.format(duration[i].strip())

dur_hours = []
dur_minutes = []  
dur_seconds = []
for i in range(len(duration)) :
    dur_minutes.append(int(duration[i].split()[0][:-1])*60 + int(duration[i].split()[1][:-1]))
    dur_seconds.append(int(duration[i].split()[0][:-1])*60*60 + int(duration[i].split()[1][:-1])*60)
df_test["Duration_minutes"] = dur_minutes
df_test.drop(labels = 'Duration', axis = 1, inplace = True)

In [88]:
# Training Set
df_train['Depart_Time_Hour'] = pd.to_datetime(df_train.Dep_Time).dt.hour
df_train['Depart_Time_Minutes'] = pd.to_datetime(df_train.Dep_Time).dt.minute
df_train.drop(labels = 'Dep_Time', axis = 1, inplace = True)
df_train['Arr_Time_Hour'] = pd.to_datetime(df_train.Arrival_Time).dt.hour
df_train['Arr_Time_Minutes'] = pd.to_datetime(df_train.Arrival_Time).dt.minute
df_train.drop(labels = 'Arrival_Time', axis = 1, inplace = True)

# Prediction Set
df_test['Depart_Time_Hour'] = pd.to_datetime(df_test.Dep_Time).dt.hour
df_test['Depart_Time_Minutes'] = pd.to_datetime(df_test.Dep_Time).dt.minute
df_test.drop(labels = 'Dep_Time', axis = 1, inplace = True)
df_test['Arr_Time_Hour'] = pd.to_datetime(df_test.Arrival_Time).dt.hour
df_test['Arr_Time_Minutes'] = pd.to_datetime(df_test.Arrival_Time).dt.minute
df_test.drop(labels = 'Arrival_Time', axis = 1, inplace = True)

In [89]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Price,isWeekend,Day_of_Week,Day_Of_Journey,Month_of_Journey,Duration_minutes,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,IndiGo,24/03/2019,Banglore,Delhi,BLR ? DEL,0,No info,3897,1,Sunday,24,3,170,22,20,1,10
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2,No info,7662,0,Wednesday,1,5,445,5,50,13,15
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,2,No info,13882,1,Sunday,9,6,1140,9,25,4,25
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,1,No info,6218,1,Sunday,12,5,325,18,5,23,30
4,IndiGo,01/03/2019,Banglore,Delhi,BLR ? NAG ? DEL,1,No info,13302,0,Friday,1,3,285,16,50,21,35


# Model Selection :
There are around 60+ predictive modeling algorithms that are available to choose from. We must have a good understanding of our data and the desired solution we are looking for, to narrow down our model selection. Here our goal is to predict the the average price by developing a supervised machine learning model and we want to identify a relation between our output (price) and remaining independent features . This scenario is a classic example of Regression (price).

We are provided with 10683 rows of the labeled dataset tagged with the output column “Price” to train our model. Whenever the dataset is labeled and output feature is known, we opt for the Supervised Learning Machine Learning technique. So, our use-case is a Supervised Learning plus Regression problem. Based on these criteria we can narrow down our choice of models to a few:

Linear Regression

DecisionTreeRegressor()

RandomForestRegressor()



# Identifying features:
Here we are dividing our data into a dependent feature that is object as y and independent features as x.

In [90]:
# We created X and y columns for training dataset while y column for dataset to predict will
# be generated by our model in final step
X = df_train.drop(["Price"], axis=1)
y = np.log1p(df_train["Price"])
# y = df_train["Price"]
X_test = df_test

In [92]:
#Separate categorical and numerical columns in dataframe
X_categorical = X.select_dtypes(exclude=['int', 'float'])
X_numerical = X.select_dtypes(include=['int', 'float'])

X_test_categorical = X_test.select_dtypes(exclude=['int', 'float'])
X_test_numerical = X_test.select_dtypes(include=['int', 'float'])

In [93]:
# Check if categorical columns are assigned properly
X_categorical.head()

#Create a back up copy specifically for plotting graphs
X_numerical_graph = X_numerical.copy()
X_categorical_graph = X_categorical.copy()

In [94]:
#Check if numerical columns are assigned properly
X_categorical.shape

(10462, 14)

In [96]:
#We replace values based on insights gathered from graphs generated in further steps
X_categorical["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

X_test_categorical["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

In [97]:

#We replace values based on insights gathered from graphs generated in further steps
X_categorical["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)
X_test_categorical["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)

In [98]:
#Label encode and hot encode categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_categorical = X_categorical.apply(LabelEncoder().fit_transform)
X_test_categorical = X_test_categorical.apply(LabelEncoder().fit_transform)

In [99]:
# Check the skew of all numerical features
from scipy.stats import skew
skewed_feats = X_numerical.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))


Skew in numerical features: 

There are 2 skewed numerical features to Box Cox transform


In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
cols_train = list(X_numerical.columns)
index_train = X_numerical.index.tolist()
cols_test = list(X_test_numerical.columns)
index_test = X_test_numerical.index.tolist()

# We can also use standard scaler if required...The code is commented for now
# Get column names first
X_numerical_names = X_numerical.columns
X_numerical_index = X_numerical.index
X_test_names = X_test_numerical.columns
# Create the Scaler object
# scaler = StandardScaler()
# Fit your data on the scaler object
# scaled_df = scaler.fit_transform(X_numerical)
# X_numerical = pd.DataFrame(scaled_df, columns=X_numerical_names, index=index_train)
# scaled_df_to_predict = scaler.fit_transform(X_to_predict_numerical)
# X_to_predict_numerical = pd.DataFrame(scaled_df_to_predict, columns=X_to_predict_numerical_names, index = index_test)

# mmx = MinMaxScaler()
# df_minmax = mmx.fit_transform(X_numerical)
# df_minmax_test = mmx.fit_transform(X_to_predict_numerical)
# X_numerical = pd.DataFrame(data=df_minmax,columns=cols_train,index=index_train)
# X_to_predict_numerical = pd.DataFrame(data=df_minmax_test,columns=cols_test,index=index_test)

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.5
for feat in skewed_features:
    #all_data[feat] += 1
    X_numerical[feat] = boxcox1p(X_numerical[feat], lam)
    X_test_numerical[feat] = boxcox1p(X_test_numerical[feat], lam)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
#Merge categorical and numerical columns back into respective X and X_to_predict
X = pd.concat([X_categorical, X_numerical], axis=1)
X_test = pd.concat([X_test_categorical, X_test_numerical], axis=1)

In [103]:
#Check shape of all three to verify that merge was done properly in above step
print(X.shape, X_numerical.shape, X_categorical.shape)
print(X_test.shape, X_test_numerical.shape, X_test_categorical.shape)
print(y.shape)

(10462, 16) (10462, 2) (10462, 14)
(2671, 16) (2671, 2) (2671, 14)
(10462,)


# Splitting Data:
we split our data into a test set and train set using train_test_split() which helps to better train the model and test its performance. test_size=0.15 creates a test data set with 15% of the data we have and the remaining 85% percent will be our training set to train our model. This split function returns us a pair of the train set (x_train,y_train) and a pair of the test set (x_test,y_test).

In [123]:
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size = .15 , random_state = 40)

In [124]:
model = [LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()]

for m in model:
    m.fit(x_train,y_train)
    m.score(x_train,y_train)
    predm = m.predict(x_test)
    print('error')
    print('mae',mean_absolute_error(y_test,predm))
    print('mse',mean_squared_error(y_test,predm))
    print('rmse',np.sqrt(mean_squared_error(y_test,predm)))
    print('score',r2_score(y_test,predm))
    print('\n')

error
mae 0.2694804599423165
mse 0.1109279641306858
rmse 0.33305849956229283
score 0.5599231205709251


error
mae 0.07418938930115865
mse 0.027090976677924233
rmse 0.16459336766080288
score 0.8925238322857784


error
mae 0.06718753331472718
mse 0.015870692615181176
rmse 0.12597893718864744
score 0.9370372932091436




# we can observe that Random Forest Regressor algorithem worked very well.
Accuracy score of both algorithems are as given:-

Accuracy score of Linear Regression is 55%

Accuracy score of DecisionTreeRegressor() is 89%

Accuracy score of RandomForestRegressor() is 93%

Hence we can say that out all algorithems RandomForestRegressor() is best for this problem till now however we will use cross validation to select best model.

# Model Validation


In [126]:
clf = RandomForestRegressor()
scores = cross_val_score(clf,X,y,cv=5,scoring='r2')
scores.mean()

0.9394974131806897

# Save the model using Pickle


In [127]:
# Save the model using pickle
import pickle
filename = "pickleRFRfile.pkl"
pickle.dump(clf, open(filename, 'wb'))


# Conclusion:
we achieved highest accuracy score of 93% by using Random Forest Regressor which is good for any machine learning prediction model particularly for a linear regression problem. Finally! we successfully created a Regression Machine Learning prediction model using Python and its powerful libraries which predicts the ticket price of flight.